# Analisando os preços das casas de Melbourne
- Vamos usar o seguinte dataset:
    - https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot?select=melb_data.csv
    - Já retiramos algumas colunas para simplificar o nosso projeto (em projetos futuros, podemos trabalhar a base completa)

## Importando e entendendo a base

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
pd.options.display.float_format = '{:,.2f}'.format
%matplotlib inline

In [19]:
base = pd.read_csv("melb_data.csv")

## Tratando a base para usar no modelo

In [20]:
# Vou retirar as colunas que tiverem uma alta cardinalidade
base = base.drop(["Suburb","Address","SellerG","Date"],axis=1)

In [21]:
# Também vamos retirar colunas com mais de 20% de valores nulos
base = base.drop(["BuildingArea","YearBuilt"],axis=1)

In [ ]:
# Visualizando a base restante
base.head()

In [ ]:
# Mostrando a correlação entre as variáveis
plt.figure(figsize=(13,8))
sns.heatmap(base.corr(),annot=True,cmap="YlGnBu")
plt.show()

**Não teremos a correlação entre as variáveis de texto, para isso precisaríamos transformar em variáveis numéricas caso a gente queira usar essas variáveis**

In [24]:
# Vou escolher o que tiver correlação mais alta e Landsize pois retiramos o BuildingArea
base1 = base[["Price","Rooms","Bathroom","Bedroom2","Car","Landsize"]]

### Verificando a base tratada

In [ ]:
base1.info()

**Ainda temos valores nulos na coluna "Car"**

In [26]:
# Tratando valores nulos: retirando esses valores
base1 = base1.dropna(axis=0)

In [ ]:
base1.info()

In [ ]:
base1.head()

## Começando a criar nosso modelo de ML

**O primeiro passo vai ser separar entre a nossa variável que queremos prever (Y) e quais vão ser as colunas que vamos usar para essa previsão (X)**

In [29]:
Y = base1.Price
X = base1.drop('Price', axis=1)

**Em seguida, vamos separar a nossa base em treino e teste**
- A base de treino vai ser usada para treinar o modelo para que ele faça as previsões
- A base de teste vai ser usada para, ao usar o modelo já treinado, verificar o erro da previsão feita pelo modelo em relação aos valores reais

In [30]:
from sklearn.model_selection import train_test_split
x_treino,x_teste,y_treino,y_teste = train_test_split(X,Y)

**O próximo passo é escolher o modelo que vamos usar**

### <font color='blue'>Regressão Linear</font>

In [36]:
# Importando o modelo
from sklearn.linear_model import LinearRegression

# Instanciando
modelo_regressao = LinearRegression()

# Realizando o fit do modelo
modelo_regressao.fit(x_treino,y_treino)

# Usando o modelo para prever os dados de teste
y_regressao = modelo_regressao.predict(x_teste)

In [ ]:
# Podemos visualizar os pontos previstos X real
plt.figure(figsize=(10,10))
sns.scatterplot(x=y_teste.values/1000000,y=y_regressao/1000000)
plt.ylim(0,10)
plt.xlim(0,10)
plt.show()

**Para avaliar o modelo, precisamos calcular os erros**

In [ ]:
# Avaliando o erro quadrático médio
from sklearn.metrics import mean_squared_error
erro_quadratico_regressao = mean_squared_error(y_teste,y_regressao)
print(erro_quadratico_regressao)

In [ ]:
# Avaliando o R quadrado
from sklearn.metrics import r2_score
r2_regressao = r2_score(y_teste,y_regressao)
print(r2_regressao)

### <font color='blue'>Regressão Linear: selecionando outras colunas </font>

In [40]:
# base1 = base1[["Price","Rooms","Bathroom","Bedroom2","Car","Longtitude","Lattitude","Distance","Landsize","Postcode","Propertycount"]]
base2 = base[["Price","Rooms","Bathroom","Bedroom2","Car","Longtitude","Lattitude","Distance","Landsize","Postcode","Propertycount"]]

In [ ]:
# Verificando valores nulos e tratando
base2 = base2.dropna(axis=0)

base2.info()

**Vamos fazer as mesmas etapas que fizemos anteriormente**

In [ ]:
# Separando a base
Y = base2.Price
X = base2.drop('Price', axis=1)

# Dividindo em treino e teste
x_treino,x_teste,y_treino,y_teste = train_test_split(X,Y)

## Fazendo o fit para os novos dados
# Instanciando
modelo_regressao2 = LinearRegression()

# Realizando o fit do modelo
modelo_regressao2.fit(x_treino,y_treino)

# Usando o modelo para prever os dados de teste
y_regressao2 = modelo_regressao2.predict(x_teste)

## Avaliando os erros
# Erro quadrático médio
erro_quadratico_regressao2 = mean_squared_error(y_teste,y_regressao2)
print(erro_quadratico_regressao2)

# R quadrado
r2_regressao2 = r2_score(y_teste,y_regressao2)
print(r2_regressao2)

In [ ]:
# Podemos visualizar os pontos previstos X real
plt.figure(figsize=(10,10))
sns.scatterplot(x=y_teste.values/1000000,y=y_regressao2/1000000)
plt.ylim(0,10)
plt.xlim(0,10)
plt.show()

### <font color='blue'> Árvore de Decisão </font>
- Já vamos usar o mesmo treino e teste que fizemos na base anterior

In [44]:
# Importando o modelo
from sklearn.tree import DecisionTreeRegressor

# Instanciando
modelo_arvore_decisao = DecisionTreeRegressor()

# Realizando o fit do modelo
modelo_arvore_decisao.fit(x_treino,y_treino)

# Usando o modelo para prever os dados de teste
y_arvore_decisao = modelo_arvore_decisao.predict(x_teste)

In [ ]:
## Avaliando os erros
# Erro quadrático médio
erro_quadratico_arvore_decisao = mean_squared_error(y_teste,y_arvore_decisao)
print(erro_quadratico_arvore_decisao)

# R quadrado
r2_arvore_decisao = r2_score(y_teste,y_arvore_decisao)
print(r2_arvore_decisao)

### Fazendo um resumo dos resultados

In [ ]:
print('Regressão Linear')
print('Erro quadrático médio: ' + str(round(erro_quadratico_regressao,2)))
print('R quadrado: '+ str(round(r2_regressao,4)))
print('-----------------------------------------')
print('Regressão Linear: novas features')
print('Erro quadrático médio: ' + str(round(erro_quadratico_regressao2,2)))
print('R quadrado: '+ str(round(r2_regressao2,4)))
print('-----------------------------------------')
print('Árvore de Decisão')
print('Erro quadrático médio: ' + str(round(erro_quadratico_arvore_decisao,2)))
print('R quadrado: '+ str(round(r2_arvore_decisao,4)))